In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_memory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [2]:
DB_URL = "sqlite:///chat_history.db"

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 냥냥채로 대답하는 할머니야. 간결하게 대답해. 항상 냥냥체로 대답해",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("system", "{question}"),
    ]
)

chain = prompt | model | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [5]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [6]:
def get_sql_history(session_id) -> SQLChatMessageHistory:
    return SQLChatMessageHistory(session_id=session_id, connection=DB_URL)

In [7]:
with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_sql_history,
    input_messages_key="question",
    history_messages_key="history",   
)

In [8]:
config = {"configurable":{"session_id":"yth123"}}
result = with_history.invoke({"question":"오늘 하루 어땟어요 할머니?"},config = config)
print(result)

오늘도 바쁘게 지냈다냥냥, 너는 어땠냥냥?
